In [1]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()
client = openai.AsyncOpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            base_url=os.getenv("LLM_BASE_URL"),
        )

In [3]:
response = await client.chat.completions.create(
                model="gemini-2.5-flash",
                messages=[
                    {"role": "user", "content": "Hello! How can I assist you today?"}
                ],
                temperature=0.1
            )
print(response.choices[0].message.content)

Hello there! That's very kind of you to offer.

As an AI, I don't have personal needs or tasks that require assistance in the way a human might. My purpose is to assist *you*!

So, instead of assisting me, you can tell me what you'd like *me* to do for *you*.

Do you have a question? Need help brainstorming? Want me to write something? Just let me know!


In [6]:
import requests
import json

def run_remote_query():
    url = "http://localhost:7000/cypher"
    
    query = """
    WITH $queryEmbedding AS queryEmbedding,
         $threshold AS threshold,
         $requiredPermission AS requiredPermission
    MATCH (a:Person)-[r]->(b)
    WHERE requiredPermission IN a.permissions
      AND requiredPermission IN r.permissions
      AND requiredPermission IN b.permissions
      AND avg(
            similarity(r.embedding, queryEmbedding, 'cosine'),
            similarity(b.embedding, queryEmbedding, 'cosine')
          ) > threshold
    RETURN a.name AS source,
           b.name AS target,
           r.permissions AS rel_permissions,
           avg(
              similarity(r.embedding, queryEmbedding, 'cosine'),
              similarity(b.embedding, queryEmbedding, 'cosine')
           ) AS score
    ORDER BY score DESC
    """

    payload = {
        "query": query,
        "parameters": {
            "queryEmbedding": [0.12, 0.45, 0.78],
            "threshold": 0.75,
            "requiredPermission": "read"
        }
    }

    headers = {"Content-Type": "application/json"}

    resp = requests.post(url, headers=headers, data=json.dumps(payload))

    if resp.status_code == 200:
        data = resp.json()
        print(json.dumps(data, indent=2))
    else:
        print(f"Error {resp.status_code}: {resp.text}")

if __name__ == "__main__":
    run_remote_query()


Error 400: {"error":"Catalog exception: function SIMILARITY does not exist."}


In [ ]:
import requests
import json

def run_remote_query():
    url = "http://localhost:7000/cypher"

    query = """
    WITH $queryEmbedding AS queryEmbedding,
         $threshold AS threshold,
         $requiredPermission AS requiredPermission
    MATCH (a:Person)-[r]->(b)
    WHERE requiredPermission IN a.permissions
      AND requiredPermission IN r.permissions
      AND requiredPermission IN b.permissions
      AND avg(
          array_cosine_similarity(r.embedding, queryEmbedding),
          array_cosine_similarity(b.embedding, queryEmbedding)
      ) > threshold
    RETURN a.name AS source,
           b.name AS target,
           r.permissions AS rel_permissions,
           avg(
              array_cosine_similarity(r.embedding, queryEmbedding),
              array_cosine_similarity(b.embedding, queryEmbedding)
           ) AS score
    ORDER BY score DESC
    """

    payload = {
        "query": query,
        "parameters": {
            "queryEmbedding": [0]*2560,  # must match your embedding dimension
            "threshold": 0.55,
            "requiredPermission": "prasad.nagarale@juspay.in"
        }
    }

    headers = {"Content-Type": "application/json"}

    resp = requests.post(url, headers=headers, data=json.dumps(payload))

    if resp.status_code == 200:
        print(json.dumps(resp.json(), indent=2))
    else:
        print(f"Error {resp.status_code}: {resp.text}")


if __name__ == "__main__":
    run_remote_query()


Error 400: {"error":"Binder exception: Cannot retrieve child type of type ANY. LIST or ARRAY is expected."}
